In [1]:
import pandas as pd
pd_data = pd.read_csv('./train.csv')

/home/digitalstorm/miniconda3/envs/nvidia-dsd-v1.2/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (47) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
pd_data = pd_data[['PlayerHeight', 'PlayerWeight', 'DefendersInTheBox', 'Distance', 'Quarter',
                  'HomeScoreBeforePlay', 'VisitorScoreBeforePlay', 'NflId', 'GameId',
                  'YardLine', 'Down', 'Temperature', 'Yards']]

In [3]:
def height_to_numerical(height):
    feet = height.split('-')[0]
    inches = height.split('-')[1]
    return int(feet)*12 + int(inches)

In [4]:
pd_data['PlayerHeight'] = pd_data['PlayerHeight'].apply(height_to_numerical)

In [5]:
#2x Dataset size
pd_data = pd.concat([pd_data, pd_data], ignore_index=True)
#4x Dataset size
pd_data = pd.concat([pd_data, pd_data], ignore_index=True)
#8x Dataset size
pd_data = pd.concat([pd_data, pd_data], ignore_index=True)
#16x Dataset size
pd_data = pd.concat([pd_data, pd_data], ignore_index=True)

In [6]:
pd_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8156192 entries, 0 to 8156191
Data columns (total 13 columns):
PlayerHeight              int64
PlayerWeight              int64
DefendersInTheBox         float64
Distance                  int64
Quarter                   int64
HomeScoreBeforePlay       int64
VisitorScoreBeforePlay    int64
NflId                     int64
GameId                    int64
YardLine                  int64
Down                      int64
Temperature               float64
Yards                     int64
dtypes: float64(2), int64(11)
memory usage: 808.9 MB


In [7]:
import cudf
cudf_data = cudf.from_pandas(pd_data)
print(type(cudf_data))

<class 'cudf.dataframe.dataframe.DataFrame'>


In [8]:
cleaned_data = pd_data.to_csv('./cleaned_data.csv')
import dask_cudf
dask_data = dask_cudf.read_csv('./cleaned_data.csv')
print(type(dask_data))

<class 'dask_cudf.core.DataFrame'>


In [9]:
%%timeit
pd_data['PlayerHeight'].mean()

29.1 ms ± 105 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [10]:
%%timeit
cudf_data['PlayerHeight'].mean()

466 µs ± 2.04 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [11]:
%%timeit
dask_data['PlayerHeight'].mean()

1.46 ms ± 1.98 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [12]:
%%timeit
pd_data['DefendersInTheBox_vs_Distance'] = pd_data['DefendersInTheBox'] / pd_data['Distance']

36.9 ms ± 66.6 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [13]:
%%timeit
cudf_data['DefendersInTheBox_vs_Distance'] = cudf_data['DefendersInTheBox'] / cudf_data['Distance']

3.1 ms ± 10.4 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [14]:
%%timeit
dask_data['DefendersInTheBox_vs_Distance'] = dask_data['DefendersInTheBox'] / dask_data['Distance']

48.4 ms ± 67.3 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [15]:
%%timeit
pd_data['BMI'] = 703 * (pd_data['PlayerWeight']/pd_data['PlayerHeight']**2)

64.3 ms ± 112 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
%%timeit
cudf_data['BMI'] = 703 * (cudf_data['PlayerWeight']/cudf_data['PlayerHeight']**2)

15.7 ms ± 501 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [17]:
%%timeit
pd_data['Margin'] = pd_data['HomeScoreBeforePlay'] - pd_data['VisitorScoreBeforePlay']

32 ms ± 40.4 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [18]:
%%timeit
cudf_data['Margin'] = cudf_data['HomeScoreBeforePlay'] - cudf_data['VisitorScoreBeforePlay']

3.46 ms ± 7.82 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [19]:
%%timeit
pd_data['Urgency'] = pd_data['Quarter'] * pd_data['Margin']

33.6 ms ± 33.4 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [20]:
%%timeit
cudf_data['Urgency'] = cudf_data['Quarter'] * cudf_data['Margin']

3.46 ms ± 8.19 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [21]:
%%time
for col in pd_data:
    pd_data[col] = pd_data[col] / pd_data[col].max()

CPU times: user 1.28 s, sys: 372 ms, total: 1.65 s
Wall time: 1.65 s


In [22]:
%%time
for col in cudf_data:
    cudf_data[col] = cudf_data[col] / cudf_data[col].max()

CPU times: user 117 ms, sys: 52.1 ms, total: 169 ms
Wall time: 167 ms


In [23]:
pd_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8156192 entries, 0 to 8156191
Data columns (total 17 columns):
PlayerHeight                     float64
PlayerWeight                     float64
DefendersInTheBox                float64
Distance                         float64
Quarter                          float64
HomeScoreBeforePlay              float64
VisitorScoreBeforePlay           float64
NflId                            float64
GameId                           float64
YardLine                         float64
Down                             float64
Temperature                      float64
Yards                            float64
DefendersInTheBox_vs_Distance    float64
BMI                              float64
Margin                           float64
Urgency                          float64
dtypes: float64(17)
memory usage: 1.0 GB


In [24]:
import numpy as np
X = np.array(pd_data.drop(columns='Yards'))
y = pd_data['Yards']
print(y.value_counts())
y = np.array(y)

 0.020202    1051072
 0.010101     968704
 0.030303     938432
 0.040404     824032
 0.000000     755040
 0.050505     619872
 0.060606     429792
-0.010101     364672
 0.070707     320320
-0.020202     234784
 0.080808     234432
 0.090909     231616
 0.111111     136928
-0.030303     133408
 0.101010     115104
-0.040404      89056
 0.121212      83776
 0.141414      73216
 0.131313      67936
 0.151515      52800
 0.161616      48576
-0.050505      39776
 0.171717      38368
 0.181818      31680
 0.212121      28512
 0.202020      26048
 0.191919      22176
-0.060606      18656
 0.252525      13376
 0.232323      13376
              ...   
 0.404040       1056
 0.474747       1056
 0.424242       1056
 0.606061        704
 0.656566        704
 0.484848        704
 0.909091        704
 0.666667        704
 0.727273        704
 0.393939        704
 0.717172        704
 0.494949        704
 0.444444        704
 0.707071        704
 0.595960        704
-0.111111        704
 0.575758    

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [26]:
import tensorflow.keras as keras

In [27]:
model = keras.models.Sequential()
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(512, activation='relu'))
model.add(keras.layers.Dense(256, activation='relu'))
model.add(keras.layers.Dense(1, activation='tanh'))

In [28]:
from tensorflow.keras.optimizers import SGD
model.compile(optimizer=SGD(lr=0.001),
             loss='mse')

In [29]:
model.fit(X_train, y_train, epochs=10)

Train on 6524953 samples
Epoch 1/10


1271104/6524953 [====>.........................] - ETA: 9:12 - loss: 0.0043

KeyboardInterrupt: 

In [ ]:
import warnings
warnings.filterwarnings('ignore')
model.evaluate(X_test, y_test)